In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/Processed_dataset_2.csv'
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
print(df.head())


                              Functional Description  \
0                        get the role of this object   
1                       get the state of this object   
2  called by the browser or applet viewer to info...   
3  gets the accessible context associated with th...   
4  determines this applet ' s context , which all...   

                   Method Name       Predicted_Method   Prefix       Infix  \
0          get accessible role                getRole      get         NaN   
1     get accessible state set               getState      get         NaN   
2                      destroy  informAppletOfReclaim  destroy         NaN   
3       get accessible context   getAccessibleContext      get  accessible   
4           get applet context       getAppletContext      NaN         NaN   

  Suffix  Prefix Probability  Infix Probability  Suffix Probability  \
0    NaN             0.87553           0.000000                 0.0   
1    NaN             0.87553           0.000000     

In [ ]:
# List of special words that should be capitalized
special_words = {"url": "URL", "utf": "UTF", "http": "HTTP", "id": "ID"}

def to_camel_case(s):
    words = s.split()
    camel_case_str = words[0].lower() + ''.join(word.capitalize() if word.lower() not in special_words else special_words[word.lower()] for word in words[1:])
    return camel_case_str

# Apply the function to the "Method Name" column
df['Mname'] = df['Method Name'].apply(to_camel_case)

# Display the first few rows to verify the changes
print(df.head())

                              Functional Description  \
0                        get the role of this object   
1                       get the state of this object   
2  called by the browser or applet viewer to info...   
3  gets the accessible context associated with th...   
4  determines this applet ' s context , which all...   

                   Method Name       Predicted_Method   Prefix       Infix  \
0          get accessible role                getRole      get         NaN   
1     get accessible state set               getState      get         NaN   
2                      destroy  informAppletOfReclaim  destroy         NaN   
3       get accessible context   getAccessibleContext      get  accessible   
4           get applet context       getAppletContext      NaN         NaN   

  Suffix  Prefix Probability  Infix Probability  Suffix Probability  \
0    NaN             0.87553           0.000000                 0.0   
1    NaN             0.87553           0.000000     

In [ ]:
pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.7 MB/s eta 0:00:00


In [ ]:
import Levenshtein
def calculate_edit_distance(row):
    mname = str(row['Mname']) if pd.notnull(row['Mname']) else ''
    predicted_method = str(row['Predicted_Method']) if pd.notnull(row['Predicted_Method']) else ''
    return Levenshtein.distance(mname, predicted_method)


In [ ]:
df_subset = df.iloc[:5000].copy()
df_subset['Edit_Score'] = df_subset.apply(calculate_edit_distance, axis=1)
output_csv_file_path = '/content/drive/MyDrive/Method/EnglishMethod/PromptProcessDataset/Process_Dataset3.csv'
df_subset.to_csv(output_csv_file_path, index=False)

In [ ]:
df_subset.head(5)

,Functional Description,Method Name,Predicted_Method,Prefix,Infix,Suffix,Prefix Probability,Infix Probability,Suffix Probability,Pivot,Mname,Edit_Score
0,get the role of this object,get accessible role,getRole,get,NaN,NaN,0.87553,0.000000,0.0,"{'get': ('get', 0.6809051036834717), 'accessib...",getAccessibleRole,10
1,get the state of this object,get accessible state set,getState,get,NaN,NaN,0.87553,0.000000,0.0,"{'get': ('get', 0.6631520986557007), 'accessib...",getAccessibleStateSet,13
2,called by the browser or applet viewer to info...,destroy,informAppletOfReclaim,destroy,NaN,NaN,0.87500,0.000000,0.0,"{'destroy': ('allocated', 0.47457969188690186)}",destroy,20
3,gets the accessible context associated with th...,get accessible context,getAccessibleContext,get,accessible,NaN,0.87553,0.833333,0.0,"{'get': ('gets', 0.610834002494812), 'accessib...",getAccessibleContext,0
4,"determines this applet ' s context , which all...",get applet context,getAppletContext,NaN,NaN,NaN,0.00000,0.000000,0.0,"{'get': ('query', 0.4418052136898041), 'apple'...",getAppletContext,0


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Create a function to generate the description for each row
def generate_description(row):
    description = f'1) Functional Description: "{row["Functional Description"]}" Method Name: "{row["Mname"]}". '
    description += f'In this example'
    if pd.notna(row["Prefix"]):
        description += f'"{row["Prefix"]}" is most likely to be the prefix with a probability of {row["Prefix Probability"]:.5f}%. '
    if pd.notna(row["Infix"]):
        description += f'"{row["Infix"]}" is the Infix with a probability of {row["Infix Probability"]:.5f}%. '
    if pd.notna(row["Suffix"]):
        description += f'"{row["Suffix"]}" is the Suffix with a probability of {row["Suffix Probability"]:.5f}% in the method name. '

    description += f'The semantic similarity between description and method token is "{row["Pivot"]}". '
    description += f'The base LLM predicted the method name "{row["Predicted_Method"]}", which has an edit distance score of {row["Edit_Score"]} compared to the ground truth "{row["Mname"]}".'
    print(description)
    return description

# Apply the function to each row and create a new column "Example Description"
df_subset['Example Description'] = df_subset.apply(generate_description, axis=1)

# Display the first few rows to verify the changes
print(df_subset[['Functional Description', 'Mname', 'Example Description']].head())

# Save the modified DataFrame to a new CSV file (optional)
output_csv_file_path = 'output_file_with_descriptions.csv'
df_subset.to_csv(output_csv_file_path, index=False)

Streaming output truncated to the last 5000 lines.
1) Functional Description: "determines if this applet is active" Method Name: "isActive". More on, semantic similarity between description and method token is "{'is': ('determines', 0.4727569818496704), 'active': ('active', 0.6838831901550293)}". The base LLM predicted the method name "isActive", which has an edit distance score of 0 compared to the ground truth "isActive".
1) Functional Description: "indicates if this container is a validate root" Method Name: "isValidateRoot". More on, semantic similarity between description and method token is "{'is': ('is', 0.5056452751159668), 'valid': ('valid', 0.7698918581008911), '##ate': ('a', 0.5527607798576355), 'root': ('root', 0.8439710140228271)}". The base LLM predicted the method name "isValidateRoot", which has an edit distance score of 0 compared to the ground truth "isValidateRoot".
1) Functional Description: "get an audio clip from the given url" Method Name: "newAudioClip". In this

In [ ]:
print(df_subset['Example Description'])

0       1) Functional Description: "get the role of th...
1       1) Functional Description: "get the state of t...
2       1) Functional Description: "called by the brow...
3       1) Functional Description: "gets the accessibl...
4       1) Functional Description: "determines this ap...
                              ...                        
4995    1) Functional Description: "compares two remot...
4996    1) Functional Description: "returns a hashcode...
4997    1) Functional Description: "returns the stub f...
4998    1) Functional Description: "processes a method...
4999    1) Functional Description: "deprecated 1 2 sty...
Name: Example Description, Length: 5000, dtype: object
